In [191]:
import json
import json
from random import choice
from nltk.tokenize import word_tokenize
from jellyfish import levenshtein_distance
import json
from pymorphy2 import MorphAnalyzer
from random import choice
from collections import Counter, defaultdict


In [192]:
word_accents_dict = json.loads(open("classic-phonetic-templates/words_accent.json").read())

In [193]:
len(word_accents_dict)

98353

In [28]:
set([d['poet_id'] for d in data])

{'belyi', 'blok', 'esenin', 'mayakovskiy', 'pushkin', 'tyutchev'}

In [9]:




def get_vowel_count(word):
    vowels = "уеыаоэёяию"
    vowel_count = 0

    for ch in word:
        if ch in vowels:
            vowel_count += 1

    return vowel_count


def get_accent(word):
    if word in word_accents_dict:
        return word_accents_dict[word]

    vowel_count = get_vowel_count(word)
    return (vowel_count + 1) // 2


def get_phoneme(word):
    word = word.lower()

    word_end = word[-3:]
    vowel_count = get_vowel_count(word)
    accent = get_accent(word)

    return word_end, vowel_count, accent


def text2template(text):
    lines = text.split("\n")

    for i, line in enumerate(lines):
        tokens = word_tokenize(line)
        tokens = [(get_phoneme(token)) for token in tokens if token.isalpha()]

        lines[i] = tokens

    return lines


def find_token(phoneme, corpus):
    word_end, vowel_count, accent = phoneme
    min_dist = 100000
    result_tokens = []

    key = "%s-%s" % (vowel_count, accent)
    if key not in corpus:
        key = choice([i for i in corpus])

    mini_corpus = corpus[key]

    for token, token_phoneme in mini_corpus.items():
        dist = levenshtein_distance(word_end, token_phoneme)

        if dist < min_dist:
            min_dist = dist
            result_tokens = [token]
        elif dist == min_dist:
            result_tokens += [token]

    return choice(result_tokens)


In [13]:


def get_seed_type(seed):
    seed_keywords = json.loads(open("classic-phonetic-templates/seed_keywords.json").read())

    for seed_type in seed_keywords:
        for keyword in seed_keywords[seed_type]:
            if keyword in seed:
                break
        else:
            continue
        break
    else:
        seed_type = "other"

    return seed_type


def load_poem(author_id):
    poems = json.loads(open("classic-phonetic-templates/styles.json").read())
    poems = poems[author_id]
    poem = choice(poems)

    return poem


def generate_text(template, corpus):
    text = []
    for temp_line in template:
        row = []
        for t in temp_line:
            row.append(find_token(t, corpus))
        text.append(" ".join(row))

    return "\n".join(text)


def load_corpus(seed_type):
    corpus_text = open("classic-phonetic-templates/corpus/%s.txt" % seed_type).read()

    tokens = corpus_text.split()
    corpus = dict()

    for token in tokens:
        word_end, vowel_count, accent = get_phoneme(token)

        key = "%s-%s" % (vowel_count, accent)
        w = corpus.get(key, dict())

        w[token] = word_end
        corpus[key] = w

    return corpus


def generate_poem(author_id, seed):
    seed_type = get_seed_type(seed)
    poem = load_poem(author_id)
    template = text2template(poem)
    corpus = load_corpus(seed_type)
    poem = generate_text(template, corpus)

    return poem


In [29]:
seed_type = get_seed_type(seed)
poem = load_poem(author_id)
template = text2template(poem)
corpus = load_corpus(seed_type)
poem = generate_text(template, corpus)



In [31]:
template


[[('я', 1, 1),
  ('вас', 1, 1),
  ('бил', 2, 1),
  ('овь', 2, 2),
  ('еще', 2, 2),
  ('ыть', 1, 1),
  ('жет', 2, 1)],
 [('в', 0, 0),
  ('уше', 2, 1),
  ('оей', 2, 1),
  ('сла', 3, 2),
  ('не', 1, 1),
  ('сем', 2, 2)],
 [('но', 1, 1),
  ('сть', 1, 1),
  ('она', 2, 2),
  ('вас', 1, 1),
  ('ьбе', 2, 1),
  ('не', 1, 1),
  ('жит', 3, 2)],
 [('я', 1, 1),
  ('не', 1, 1),
  ('очу', 2, 1),
  ('ить', 3, 2),
  ('вас', 1, 1),
  ('чем', 2, 1)],
 [('я', 1, 1), ('вас', 1, 1), ('бил', 2, 1), ('вно', 3, 2), ('жно', 4, 2)],
 [('то', 1, 1), ('тью', 3, 2), ('то', 1, 1), ('тью', 3, 2), ('мим', 2, 1)],
 [('я', 1, 1),
  ('вас', 1, 1),
  ('бил', 2, 1),
  ('так', 1, 1),
  ('нно', 3, 2),
  ('так', 1, 1),
  ('жно', 2, 1)],
 [('как', 1, 1),
  ('дай', 1, 1),
  ('вам', 1, 1),
  ('бог', 1, 1),
  ('мой', 3, 2),
  ('ыть', 1, 1),
  ('гим', 2, 1)]]

In [2]:
import gensim
from nltk.tokenize import WordPunctTokenizer
import re

In [3]:
wiki = open('../wiki_texts.txt')

In [4]:
wiki_texts = []
for i,text in enumerate(wiki):
    if i > 400000:
        break
    words = [w.split('_')[0] for w in text.rstrip('\n').split()[:100]]
    wiki_texts.append(words)

In [5]:
wiki_texts[:2]

[['литва',
  'официальный',
  'название',
  'лить',
  'вскай',
  'респу',
  'блик',
  'государство',
  'расположить',
  'северный',
  'европа',
  'один',
  'страна',
  'балтия',
  'столица',
  'страна',
  'вильнюс',
  'площадь',
  'км²',
  'протяжённость',
  'север',
  'юг',
  'километр',
  'запад',
  'восток',
  'километр',
  'население',
  'составлять',
  'человек',
  'показатель',
  'являться',
  'крупный',
  'прибалтийский',
  'государство',
  'иметь',
  'выход',
  'балтийский',
  'море',
  'расположить',
  'восточный',
  'побережье',
  'береговой',
  'линия',
  'составлять',
  'всего',
  'километр',
  'наименьший',
  'показатель',
  'прибалтийский',
  'государство',
  'север',
  'граничить',
  'латвия',
  'юго',
  'восток',
  'белоруссия',
  'юго',
  'запад',
  'польша',
  'калининградский',
  'область',
  'россия',
  'член',
  'оон',
  'год',
  'ес',
  'нато',
  'год',
  'входить',
  'шенгенский',
  'зона',
  'еврозона',
  'географический',
  'дать',
  'карта',
  'литва',
  'пове

In [68]:
tokenizer = WordPunctTokenizer().tokenize

In [64]:
texts = [d['content'] for d in data]

In [69]:
tokenizer(texts[1])

['Песнь',
 'первая',
 '\\',
 'nСвятой',
 'монах',
 ',',
 'грехопадение',
 ',',
 'юбка',
 '\\',
 'nХочу',
 'воспеть',
 ',',
 'как',
 'дух',
 'нечистый',
 'Ада',
 '\\',
 'nОседлан',
 'был',
 'брадатым',
 'стариком',
 ';\\',
 'nКак',
 'овладел',
 'он',
 'черным',
 'клобуком',
 ',\\',
 'nКак',
 'он',
 'втолкнул',
 'Монаха',
 'грешных',
 'в',
 'стадо',
 '.\\',
 'nПевец',
 'любви',
 ',',
 'фернейской',
 'старичок',
 ',\\',
 'nК',
 'тебе',
 ',',
 'Вольтер',
 ',',
 'я',
 'ныне',
 'обращаюсь',
 '.\\',
 'nКуда',
 ',',
 'скажи',
 ',',
 'девался',
 'твой',
 'смычок',
 ',\\',
 'nКоторым',
 'я',
 'в',
 'Жан',
 'д',
 "'",
 'Арке',
 'восхищаюсь',
 ',\\',
 'nГде',
 'кисть',
 'твоя',
 ',',
 'скажи',
 ',',
 'ужели',
 'ввек',
 '\\',
 'nИх',
 'ни',
 'один',
 'не',
 'найдет',
 'человек',
 '?\\',
 'nВольтер',
 '!',
 'Султан',
 'французского',
 'Парнасса',
 ',\\',
 'nЯ',
 'не',
 'хочу',
 'седлать',
 'коня',
 'Пегаса',
 ',\\',
 'nЯ',
 'не',
 'хочу',
 'из',
 'муз',
 'наделать',
 'дам',
 ',\\',
 'nНо',
 'дай',
 

In [139]:
sents = []
for text in texts:
    if not re.search('[a-zA-Z]', text.replace('\\n', '')):
#         lines = text.split('\\n')
#         for line in lines[1:]:
        words = [x.lower() for x in tokenizer(text.replace('\\n', '\n')) if x.isalpha()]
        sents.append(words)
            

In [114]:
stihi_ru = open('stihi_ru/corpus_stihi_ru.txt').read()

In [119]:
shr = []
for line in stihi_ru:
    if not re.search('[a-zA-Z]', line.replace('\\n', '')):
        words = [x.lower() for x in tokenizer(line) if x.isalpha()]
        if words:
            shr.append(words)

In [120]:
shr[:10]

[['когда', 'февраль', 'ещё', 'не', 'спорил', 'с', 'мартом'],
 ['кто', 'холодней', 'и', 'чья', 'больней', 'беда'],
 ['не', 'рисовала', 'контурные', 'карты'],
 ['земная', 'тяжесть', 'на', 'ногах'],
 ['когда'],
 ['вина', 'не', 'копошилась', 'за', 'грудиной'],
 ['был', 'впереди', 'всегда', 'весёлый', 'май'],
 ['и', 'пахла', 'молоком', 'макушка', 'сына'],
 ['и', 'сколько', 'хочешь', 'столько', 'обнимай'],
 ['казалось', 'всё', 'закончится', 'не', 'скоро']]

In [79]:
sents[:10]

[['святой', 'монах', 'грехопадение', 'юбка'],
 ['хочу', 'воспеть', 'как', 'дух', 'нечистый', 'ада'],
 ['оседлан', 'был', 'брадатым', 'стариком'],
 ['как', 'овладел', 'он', 'черным', 'клобуком'],
 ['как', 'он', 'втолкнул', 'монаха', 'грешных', 'в', 'стадо'],
 ['певец', 'любви', 'фернейской', 'старичок'],
 ['к', 'тебе', 'вольтер', 'я', 'ныне', 'обращаюсь'],
 ['куда', 'скажи', 'девался', 'твой', 'смычок'],
 ['которым', 'я', 'в', 'жан', 'д', 'арке', 'восхищаюсь'],
 ['где', 'кисть', 'твоя', 'скажи', 'ужели', 'ввек']]

In [145]:
vm = open('lt1.txt', encoding='cp1251').read().splitlines()

In [147]:
pars = []
for par in vm:
    words = [x.lower() for x in tokenizer(par) if x.isalpha()]
    if words:
        pars.append(words)

In [159]:
wiki_text = []
for line in open('data/sentences_mln.txt'):
    line = [word for word in word_tokenize(line.rstrip('\n').lower()) if word.isalnum()]
    wiki_text.append(line)

KeyboardInterrupt: 

In [215]:
del (wiki_text)

In [161]:
ft = gensim.models.FastText(wiki_text, size=300, min_n=4,  
                            window=10, sg=1, max_vocab_size=300000)

In [77]:
ft.save('ft_300_100k')

In [11]:
ft = gensim.models.FastText.load('ft_300_100k')

In [162]:
ft.wv.most_similar('судdtnthьба')

[('судьбах', 0.7456399202346802),
 ('судьбы', 0.6444624066352844),
 ('судьбу', 0.6284289956092834),
 ('судьбе', 0.6257268190383911),
 ('судьбой', 0.5634428262710571),
 ('дальнейшая', 0.5511305332183838),
 ('загадка', 0.5018789172172546),
 ('жизнь', 0.47880128026008606),
 ('неизвестна', 0.47170332074165344),
 ('персонификация', 0.4667775630950928)]

In [194]:
morph = MorphAnalyzer()

In [195]:
morph.parse(token)[0].tag.POS

'NOUN'

In [217]:
int_sents = open('data/sentences_mln_non_wiki.txt')

# end2word = defaultdict(Counter)

for sent in int_sents:
    tokens = list(filter(lambda x: not bool(re.search('[^а-яё]', x)) ,
                         word_tokenize(sent.lower().rstrip('\n'))))
    for token in tokens:
        pos = morph.parse(token)[0].tag.POS
        vowels = [(i, x) for i, x in enumerate(token) if x in set('ыуаеоэюияё')]
        if vowels:
            max_v = max([v[0] for v in vowels])
            end2word[(token[max_v:], len(vowels), pos)].update([token])

In [218]:
len(end2word)

16035

In [113]:
'sdsda'[4:]

'a'

In [198]:
end2word.keys()

dict_keys([('еб', 3, 'NOUN'), ('арл', 5, 'NOUN'), ('айдг', 1, 'NOUN'), ('ют', 2, 'VERB'), ('еньш', 2, 'NOUN'), ('эйбл', 1, 'NOUN'), ('ез', 3, 'NOUN'), ('оль', 3, 'NOUN'), ('эрд', 1, 'NOUN'), ('илл', 3, 'NOUN'), ('ёр', 3, 'NOUN'), ('е', 10, 'ADJF'), ('о', 1, None), ('ав', 2, 'GRND'), ('об', 1, 'NOUN'), ('аж', 6, 'NOUN'), ('ерв', 2, 'NOUN'), ('ерс', 6, 'NOUN'), ('уц', 2, 'NOUN'), ('осць', 3, 'NOUN'), ('етът', 1, 'NOUN'), ('ассп', 1, 'NOUN'), ('ыг', 3, 'NOUN'), ('ис', 1, 'INTJ'), ('юс', 1, None), ('ифт', 3, 'NOUN'), ('о', 8, 'PRTF'), ('е', 1, 'ADVB'), ('ан', 3, 'NOUN'), ('ельств', 4, 'NOUN'), ('альд', 4, 'NOUN'), ('эвт', 2, 'NOUN'), ('ат', 9, 'PRTS'), ('ем', 1, 'PRCL'), ('айш', 3, 'NOUN'), ('ольн', 1, None), ('ейчс', 2, 'NOUN'), ('енд', 5, 'NOUN'), ('яц', 2, 'NOUN'), ('ызь', 1, 'NOUN'), ('ецб', 1, None), ('урж', 1, 'NOUN'), ('ип', 2, 'NOUN'), ('анк', 3, 'NOUN'), ('ёл', 1, 'VERB'), ('ич', 2, 'NOUN'), ('ельв', 4, 'NOUN'), ('е', 9, 'ADVB'), ('осзл', 1, 'ADJS'), ('ый', 4, 'NOUN'), ('имсх', 2,

In [15]:
data = json.loads(open('classic_poems.json').read())

In [12]:
c = Counter()
for d in data:
    c.update([d['poet_id']])

In [13]:
c

Counter({'belyi': 548,
         'blok': 1077,
         'esenin': 442,
         'mayakovskiy': 84,
         'pushkin': 479,
         'tyutchev': 414})

In [19]:
titles_pushkin = [d['title'] for d in data
                 ]

In [55]:
titles_pushkin.index('"Отговорила роща золотая…"')

597

In [ ]:
pushkin_best = [294, 385, 308]
esenin_best = [574, 582, 597]

In [80]:
mayak = [data[i]['content'].split('\\n') for i in range(len(data)) if data[i]['poet_id'] == 'mayakovskiy']
pushkin = [data[i]['content'].split('\\n') for i in range(len(data)) if data[i]['poet_id'] == 'pushkin']

In [43]:
lines = data[385]['content'].split('\\n')

In [30]:
word_tokenize(lines[1])

['В', 'одежде', 'белой', 'над', 'волнами', ',']

In [65]:
mayak[0]

['Влас Прогулкин —',
 'милый мальчик,',
 'спать ложился,',
 'взяв журнальчик.',
 'Все в журнале',
 'интересно.',
 '– Дочитаю весь,',
 'хоть тресну!',
 '– Ни отец его,',
 'ни мать',
 'не могли',
 'заставить спать.',
 'Засыпает на рассвете,',
 'скомкав',
 'ёрзаньем',
 'кровать,',
 'в час,',
 'когда',
 'другие дети',
 'бодро',
 'начали вставать.',
 'Когда',
 'другая детвора',
 'чаёвничает, вставши,',
 'отец',
 'орёт ему:',
 '– Пора!\xa0–',
 'Он —',
 'одеяло на уши.',
 'Разошлись',
 'другие',
 'в школы,\xa0–',
 'Влас',
 'у крана',
 'полуголый —',
 'не дремалось в школе чтоб,',
 'моет нос',
 'и мочит лоб.',
 'Без чаю',
 'и без калача',
 'выходит,',
 'еле волочась.',
 'Пошагал',
 'и встал разиней:',
 'вывеска на магазине.',
 'Грамота на то и есть!',
 'Надо',
 'вывеску',
 'прочесть!',
 'Прочёл',
 'с начала',
 'буквы он,',
 'выходит:',
 '„Куафёр Симон".',
 'С конца прочёл',
 'знаток наук, —',
 '„Номис"\xa0выходит',
 '„рёфаук".',
 'Подумавши',
 'минуток пять,',
 'Прогулкин',
 'двинулся опять.',

In [40]:
end2word.get(('скале'[-3:], vowels), None).keys()

dict_keys(['чале', 'але', 'скале', 'кале', 'бале', 'шкале', 'мале', 'вале', 'зале', 'дале', 'хвале', 'шквале', 'пале', 'гале', 'рале', 'шале', 'тале'])

In [12]:
from random import choice
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [219]:
vws = set('ыуаеоэюияё')

def get_sylabs(word):
    if len(word) <= 4:
        return ('##', 0)
    
    vowels = [(i, x) for i, x in enumerate(word) if x in set('ыуаеоэюияё')]
    if vowels:
        pos = morph.parse(word)[0].tag.POS
        max_v = max([v[0] for v in vowels])
        return (word[max_v:], len(vowels), pos)
    else:
        ('##', 0)

def string2set(line):
    seed_words = [word for word in word_tokenize(line) if word.isalnum()]
    return set(seed_words)

def replace_last_word(line, end2words, seed_words, ban=set(), threshold=0.7):
    
    seed_vecs = np.zeros((len(seed_words), 300))
    for i,word in enumerate(seed_words):
        try:
            seed_vecs[i] = ft[word]
        except KeyError:
            continue
    if np.nonzero(seed_vecs):
        vec = np.mean(seed_vecs, axis=0)
    else:
        return line
    
    if not line[-1].isalpha():
        last = line[-1]
        line = line[:-1]
    else:
        last = ''
    
    tokens = word_tokenize(line.lower())
#     s_tokens = set(tokens)
#     last_token = tokens[-1]
#     tokens = tokens[:-1]
    gen = set()
#     vowels = len([x for x in last_token if x in vws])
    candidates = [end2word.get(get_sylabs(word), None) for word in tokens]
#     new_token = end2word.get((last_token[-3:], vowels), None)
    for i in range(len(tokens)):
        if candidates[i]:
            cands = [x for x in candidates[i] if candidates[i][x] > 5]
            if not cands:
                continue
            matrix = np.zeros((len(cands),300))

            for j, word in enumerate(cands):
                try:
                    matrix[j] = ft[word]
                except KeyError:
                    continue
            sims = cosine_similarity(vec.reshape(1,-1), matrix)
            if sims.max() > threshold:
                new = cands[sims.argmax()]
                if new not in ban and new not in tokens:
                    tokens[i] = new
                    gen.add(new)
    #         new_token = choice([x[0] for x in new_token.most_common(50)])
        else:
            continue
    
    line = ' '.join(tokens) + last
    return line, gen

In [167]:
ft.most_similar('сигареты')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('сигарет', 0.895905613899231),
 ('сигарету', 0.8588488698005676),
 ('пачки', 0.6062952280044556),
 ('кареты', 0.6051194667816162),
 ('спички', 0.5991291403770447),
 ('конфеты', 0.5813184976577759),
 ('табачные', 0.5733342170715332),
 ('бочки', 0.5658065676689148),
 ('продавцы', 0.5656902194023132),
 ('напитки', 0.5640106797218323)]

In [1]:
seed = 'налоговый'
seed = string2set(seed)
lines = pushkin[385][:30]
gen = set()
for i in range(len(lines)):
    seed.update(gen)
    l, gen_ = replace_last_word(lines[i], end2word, seed, gen, 0.5)
    gen.update(gen_)
    print(l)
        
    

NameError: name 'string2set' is not defined

In [136]:
end2word[get_sylabs('сумрачной')]

Counter({'глобальной': 18,
         'джизакской': 1,
         'кананской': 1,
         'бородкой': 1,
         'подпольной': 36,
         'дымовой': 7,
         'балхашстрой': 1,
         'клеточной': 9,
         'массивной': 3,
         'баварской': 27,
         'поповой': 10,
         'царевской': 5,
         'супругой': 68,
         'борживой': 20,
         'штурмовой': 258,
         'заставой': 3,
         'спешенной': 1,
         'матильдой': 4,
         'смешанной': 39,
         'навозной': 2,
         'увязкой': 1,
         'врожденной': 2,
         'крылатой': 4,
         'предгорной': 11,
         'тбилисской': 1,
         'серийной': 2,
         'трубчатой': 2,
         'бурковой': 1,
         'позёмкой': 2,
         'новотной': 4,
         'рудяной': 1,
         'машинкой': 3,
         'январской': 2,
         'мохавской': 1,
         'откидной': 2,
         'медленной': 5,
         'валлонской': 1,
         'вершиной': 13,
         'усамой': 1,
         'вальмовой': 1,
    

In [37]:
'утром'[-3:]

'ром'

In [22]:
replace_last_word(lines[0], end2word, 'налоговый')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


'предплечий'

In [41]:
lines[0]

'Ты видел деву на скале'

In [62]:
[x for x in titles_pushkin if x and 'нате' in x.lower()]

['"Нате, возьмите, лопайте…"',
 '«В темной комнате ты обесчещена…»',
 '«Ты в комнате один сидишь…»']

In [21]:
print(data[1]['content'].split('\\n'))

['Песнь первая', 'Святой монах, грехопадение, юбка', 'Хочу воспеть, как дух нечистый Ада', 'Оседлан был брадатым стариком;', 'Как овладел он черным клобуком,', 'Как он втолкнул Монаха грешных в стадо.', 'Певец любви, фернейской старичок,', 'К тебе, Вольтер, я ныне обращаюсь.', 'Куда, скажи, девался твой смычок,', "Которым я в Жан д'Арке восхищаюсь,", 'Где кисть твоя, скажи, ужели ввек', 'Их ни один не найдет человек?', 'Вольтер! Султан французского Парнасса,', 'Я не хочу седлать коня Пегаса,', 'Я не хочу из муз наделать дам,', 'Но дай лишь мне твою златую лиру,', 'Я буду с ней всему известен миру.', 'Ты хмуришься и говоришь: не дам.', 'А ты поэт, проклятый Аполлоном,', 'Испачкавший простенки кабаков,', 'Под Геликон упавший в грязь с Вильоном', 'Не можешь ли ты мне помочь, Барков?', 'С усмешкою даешь ты мне скрыпицу,', 'Сулишь вино и музу пол-девицу:', '«Последуй лишь примеру моему».\xa0—', 'Нет, нет, Барков! скрыпицы не возьму,', 'Я стану петь, что в голову придется,', 'Пусть как-нибуд

In [23]:
author_id = 'pushkin'
seed = "спорт"

In [24]:
print(generate_poem(author_id, seed))

я вас разбил любовь еще быть может
в душе твоей принесла не совсем
но честь она вас душе не наложит
я не хочу заставить вас плачем
я вас убил противно ненадежно
то живостью то живостью самим
я вас любил так пространно так нежно
как дай швам бог соломой быть многим
